# Training Deep Neural Networks

## Practical Guidelines

<img src="../img/default_dnn.png" width="30%">

**Additional consideratios**
- Stadardize input features
- Reuse parts of a pre-trained NN that solves a similar problem
- Use unsupervised pretraining if you have a lot of unlabeled data
- If your model self-normalizes: If it overfits the training set, then you should add alpha dropout (and always use early stopping as well). Do not use other regularization methods, or else they would break self-normalization.
- If your model cannot self-normalize (e.g., it is a recurrent net or it contains skip connections):

-- You can try using ELU (or another activation function) instead of SELU, it may perform better. Make sure to change the initialization method accord‐ ingly (e.g., He init for ELU or ReLU).

-- If it is a deep network, you should use Batch Normalization after every hidden layer. If it overfits the training set, you can also try using max-norm or l2 reg‐ ularization.

- If you need a sparse model, you can use l1 regularization (and optionally zero out the tiny weights after training). If you need an even sparser model, you can try using FTRL instead of Nadam optimization, along with l1 regularization. In any case, this will break self-normalization, so you will need to switch to BN if your model is deep.
- If you need a low-latency model (one that performs lightning-fast predictions), you may need to use less layers, avoid Batch Normalization, and possibly replace the SELU activation function with the leaky ReLU. Having a sparse model will also help. You may also want to reduce the float precision from 32-bits to 16-bit (or even 8-bits)
- If you are building a risk-sensitive application, or inference latency is not very important in your application, you can use MC Dropout to boost performance and get more reliable probability estimates, along with uncertainty estimates.



**Vanishing / Exploding Gradients Problem:** 
When the gradients become too big / too small to be propagated through the NN.
Solution: Use a BatchNormalisation after each layer, except the output.



In [ ]:
model = keras.models.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
        keras.layers.BatchNormalization(),
        keras.layers.Dense(10, activation="softmax")
])

**WAYS TO SPEED UP TRAINING**:

- Good initialisation strategy for connection weights
- Good activation function
- Batch normalisation / Gradient Clipping
- Reuse parts of a pre-trained network
- Faster optimisers

## Function & Activation
So which activation function should you use for the hidden layers of your deep neural networks? 

In general **SELU > ELU > leaky ReLU (and its variants) > ReLU > tanh > logistic.** 
- If the network’s architecture prevents it from self- normalizing, then ELU may perform better than SELU (since SELU is not smooth at z = 0). 
- If you care a lot about runtime latency, then you may prefer leaky ReLU. If you don’t want to tweak yet another hyperparameter, you may just use the default α values used by Keras (e.g., 0.3 for the leaky ReLU). 
- If you have spare time and computing power, you can use cross-validation to evaluate other activation functions, in particular RReLU if your network is over‐ fitting, or PReLU if you have a huge training set.

- To use the leaky ReLU activation function, you must create a LeakyReLU instance like this:

> leaky_relu = keras.layers.LeakyReLU(alpha=0.2)

> layer = keras.layers.Dense(10, activation=leaky_relu, kernel_initializer="he_normal")

- For PReLU, just replace LeakyRelu(alpha=0.2) with PReLU().

- For SELU activation, just set activation="selu" and kernel_initial izer="lecun_normal" when creating a layer:
> layer = keras.layers.Dense(10, activation="selu", kernel_initializer="lecun_normal")

## Gradient Clipping

To solve the issue of exploding gradients in RNNs, it's better to use Gradient Clipping because the implementation of Batch Normalisation is more tricky.
To do so, just use the clipvalue when creating an optimiser:

> optimiser = keras.optimizer.SGD(clipvalue=1.0)  #clip every component of the gradient vector to a value between –1.0 and 1.0


You can also clip the norm, instead of the value, if you notice the gradients explode (look for more info on how to do that).

## Faster Optimizers

**Momentum optimization:** you just have to set the parameter *momentum* (0.9 is usually a good value)
>> optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9) 

**Nesterov Accelerated Gradient** sometimes improves performance. It's a variation of momentum optimization. To use set *nesterov=True*
>> optmizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

**AdaGrad** this algorithm decays the learning rate, but it does it faster for steep dimensions than for dimensions with deeper slopes (known as *adaptative learning rate*).  It doesn't work well for NNs, but it's sufficient for Linear Regression.

**RMSProp** it's a variation of AdaGrad, but only accumulates gradients of recent iterations, so it performs better. The default valud of rho works well.

>> optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

**Adam** (adaptative moment estimation) it's a variation of Momentum optimisation and RMSProp. The default values work well because it's an adaptive learning algorithm.

>> optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)




## Learning Schedules

Instead of using a fixed learning rate, start with a high value and divide it by 3 until the algorithm stops diverging. 

Ps: You can set the learning schedule in the callback function. 

Other techniques:

| Learning Rate Schedule  | Use Case                                         | Implementation Example                                                                                       |
|--------------------------|-------------------------------------------------|-------------------------------------------------------------------------------------------------------------|
| **Constant Learning Rate** | Default choice for many models. Use when unsure about dynamic adjustments. | `optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)`                                                |
| **Time-based Decay**      | Reduce learning rate over time (e.g., large datasets). | `lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(initial_learning_rate=0.001, decay_steps=10000, decay_rate=0.5)`<br>`optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)` |
| **Step Decay**            | Reduce learning rate at specific epochs.       | `lr_schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay([10000, 20000], [0.001, 0.0005, 0.0001])`<br>`optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)` |
| **Exponential Decay**     | Gradually reduce the learning rate exponentially. | `lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001, decay_rate=0.96, decay_steps=10000)`<br>`optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)` |
| **Cosine Decay**          | Oscillate learning rate for cyclical patterns. | `lr_schedule = tf.keras.optimizers.schedules.CosineDecay(initial_learning_rate=0.001, decay_steps=10000)`<br>`optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)` |
| **Cosine Decay with Warm Restarts** | Reset learning rate periodically, good for cyclical tasks. | `lr_schedule = tf.keras.optimizers.schedules.CosineDecayRestarts(initial_learning_rate=0.001, first_decay_steps=1000)`<br>`optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)` |
| **Learning Rate Finder** | Find an optimal learning rate by experimenting. | Use `tf.keras.callbacks.LearningRateScheduler` with a custom function to adjust the learning rate dynamically. |
| **Reduce on Plateau**    | Automatically reduce the learning rate when a metric (e.g., loss) stops improving. | `callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)`<br>`model.fit(..., callbacks=[callback])` |

---

### Notes
- Replace `tf.keras.optimizers.Adam` with any optimizer you're using.
- Learning rate schedules help improve model performance and convergence.
- Start simple (e.g., constant learning rate) and experiment with others as needed.

## Avoiding Overfitting Through Regularisation

### L1 & L2 
- You should apply reg to each layer
- To avoid issues, you can use the second example from below.


In [ ]:
# example of l1 regularization
layer = keras.layers.Dense(100, activation="elu",
                               kernel_initializer="he_normal",
                               kernel_regularizer=keras.regularizers.l2(0.01))

In [ ]:
# example of how you can implement regularisation in all layers

from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                                activation="elu",
                                kernel_initializer="he_normal",
                                kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
         keras.layers.Flatten(input_shape=[28, 28]),
         RegularizedDense(300),
         RegularizedDense(100),
         RegularizedDense(10, activation="softmax",
                          kernel_initializer="glorot_uniform")
])

### Dropout

This is a popular technique. At every step, every neuron (excluding the output), have probability p of being dropped out. P is called the dropout rate, typically set at 50%.

- Dropout is only applied during training. *So you can't compare validation and training loss*. Make sure to evaluate training loss after training in this case.

- If you see the model is overfitting, you can increase dropout, and decrease if the other way around.

- You can also implement dropout only after the last hidden layer (this is common). 

- It can slow the training, but the performance usually pays off. 

- If you want to regularize a self-normalizing network based on the SELU activation function, use AlphaDropout.

In [ ]:
# example of dropout implementation
model = keras.models.Sequential([
         keras.layers.Flatten(input_shape=[28, 28]),
         keras.layers.Dropout(rate=0.2),
         keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
         keras.layers.Dropout(rate=0.2),
         keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
         keras.layers.Dropout(rate=0.2),
         keras.layers.Dense(10, activation="softmax")
])

### Monte-Carlo Dropout

It can be applied to a model without the need to retrain it.

- The number of samples you use (100, int the example below), is a value you can tweak. If you set too many, it will take long, and the performance might not be too good.

- What the code does is basically generating X predicitons for every instance in the test set, stacking them, and after averaging it. 

- If you're using Normalisation layers, you can't use the code as below (look for more information on how to do this)

In [ ]:
# example of Monte Carlo dropout

with keras.backend.learning_phase_scope(1): # force training mode = dropout on
    y_probas = np.stack([model.predict(X_test_scaled) for sample in range(100)])

y_proba = y_probas.mean(axis=0)


### Max-Norm Regularisation

To implement max-norm regularization in Keras, just set every hidden layer’s ker nel_constraint argument to a max_norm() constraint, with the appropriate max value, for example:

In [ ]:
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                       kernel_constraint=keras.constraints.max_norm(1.))

## NOTES

- Glorot initialization and He initialization were designed to make the output standard deviation as close as possible to the input standard deviation, at least at the beginning of training. This reduces the vanishing/exploding gradients problem.

- **All weights should be sampled independently; they should not all have the same initial value.** One important goal of sampling weights randomly is to break symmetry: if all the weights have the same initial value, even if that value is not zero, then symmetry is not broken (i.e., all neurons in a given layer are equivalent), and backpropagation will be unable to break it. Concretely, this means that all the neurons in any given layer will always have the same weights. It's like having just one neuron per layer, and much slower. It is virtually impossible for such a configuration to converge to a good solution.

- **It is perfectly fine to initialize the bias terms to zero.** Some people like to initialize them just like weights, and that's OK too; it does not make much difference.

- ReLU is usually a good default for the hidden layers, as it is fast and yields good results. Its ability to output precisely zero can also be useful in some cases (e.g., see Chapter 17). Moreover, it can sometimes benefit from optimized implementations as well as from hardware acceleration. 

- The leaky ReLU variants of ReLU can improve the model's quality without hindering its speed too much compared to ReLU. For large neural nets and more complex problems, GLU, Swish and Mish can give you a slightly higher quality model, but they have a computational cost. 

- The hyperbolic tangent (tanh) can be useful in the output layer if you need to output a number in a fixed range (by default between –1 and 1), but nowadays it is not used much in hidden layers, except in recurrent nets. 

- The sigmoid activation function is also useful in the output layer when you need to estimate a probability (e.g., for binary classification), but it is rarely used in hidden layers (there are exceptions—for example, for the coding layer of variational autoencoders;).

- The softplus activation function is useful in the output layer when you need to ensure that the output will always be positive. 

- The softmax activation function is useful in the output layer to estimate probabilities for mutually exclusive classes, but it is rarely (if ever) used in hidden layers.

- If you set the momentum hyperparameter too close to 1 (e.g., 0.99999) when using an SGD optimizer, then the algorithm will likely pick up a lot of speed, hopefully moving roughly toward the global minimum, but its momentum will carry it right past the minimum. Then it will slow down and come back, accelerate again, overshoot again, and so on. It may oscillate this way many times before converging, so overall it will take much longer to converge than with a smaller momentum value.

- One way to produce a sparse model (i.e., with most weights equal to zero) is to train the model normally, then zero out tiny weights. For more sparsity, you can apply ℓ1 regularization during training, which pushes the optimizer toward sparsity. A third option is to use the TensorFlow Model Optimization Toolkit.

- Dropout does slow down training, in general roughly by a factor of two. However, it has no impact on inference speed since it is only turned on during training. MC Dropout is exactly like dropout during training, but it is still active during inference, so each inference is slowed down slightly. More importantly, when using MC Dropout you generally want to run inference 10 times or more to get better predictions. This means that making predictions is slowed down by a factor of 10 or more.